In [1]:
def pre_process(df):
    if framework == 'spark':
        df['thread'] = df['process']

    df['runtime'] = df.end - df.start

    # Simplify the workers name
    df['worker_name'] = df.worker.map(str).map(lambda x: ''.join(x.split('-')[1:]).split('.')[0])

    # Simplify the thread number for each worker
    thread_worker = {w+'::'+str(t): i+1 
                     for w in df.worker_name.unique()
                         for i, t in enumerate(df[df.worker_name == w].thread.unique())
                     }
    df['worker_thread'] = df.worker_name.map(str) + '::' + df.thread.map(str)
    df['thread_number'] = df.worker_thread.map(lambda x: thread_worker[x])
    df['worker_thread'] = df.worker_name.map(str) + '::thread' + df.thread_number.map(str)
    df = df.sort_values(by=['worker_name','thread_number'], ascending=[False, True])
    return df

In [8]:
from bokeh.models import CustomJS, ColumnDataSource, Plot, LinearAxis, Grid
from bokeh.models.glyphs import Quad
from bokeh.models.tools import BoxZoomTool, HoverTool, PanTool, ResetTool, SaveTool, TapTool, WheelZoomTool
from bokeh.io import curdoc, output_notebook, show
from bokeh.palettes import Colorblind8
import numpy as np
import pandas as pd
# output_notebook()

col_name = ['func', 'start', 'end', 'filename', 'worker', 'thread', 'process']
framework = 'spark'
filename = '../results/inc/data-1/results-spark_inc-baseline.csv'

df = pd.read_csv(filename, header=None, names=col_name)
df = pre_process(df)  # PRE-PROCESS

### GROUP BY ###
y = 0
labels = []
for i, x in enumerate(df.groupby('worker_thread', sort=False)):
    labels.append(x[0])
    df.loc[df.index.isin(x[1].index), 'bottom'] = y - 0.5
    df.loc[df.index.isin(x[1].index), 'top'] = y + 0.5
    y += 1
### END GROUP BY ###

# Define color map for the function
for i, x in enumerate(df.func.unique()):
    df.loc[df.func == x, 'color'] = Colorblind8[i]
df['original_color'] = df['color']

source = ColumnDataSource(df)

plot = Plot()
glyph = Quad(left="start",
             right="end",
             top="top",
             bottom="bottom",
             fill_color="color",
             fill_alpha=0.66,
             line_color="color",
             line_width=0.75,)

plot.add_glyph(source, glyph)

xaxis = LinearAxis()
plot.add_layout(xaxis, 'below')

yaxis = LinearAxis()
plot.add_layout(yaxis, 'left')

plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

# Set y axis tick label
plot.yaxis.ticker = list(range(0, len(labels)))
plot.yaxis.major_label_overrides = {k: v for k, v in zip(range(0, len(labels)), labels)}

# Hover tool
hover = HoverTool(tooltips=[
    ('filename', '@filename'),
    ('worker', '@worker_thread'),
    ('function', '@func'),
    ('runtime', '@runtime{%8.3f sec}'),
    ('start time', '@start{%8.3f sec}'),
    ('end time', '@end{%8.3f sec}'),
                            ],
                         formatters={
    'runtime': 'printf',
    'start': 'printf',
    'end': 'printf',
                         },
                 )

# Tap tool custom select
cb_click = CustomJS(args=dict(source=source), code="""
    const inds = source.selected.indices;
    const d = source.data;
    
    for (var i = 0; i < d['color'].length; i++){
        d['color'][i] = d['original_color'][i]
    }
    
    if (inds.length == 0)
        return;
        
    same_file = []
    for (var i = 0; i < d['color'].length; i++){
        if (d['filename'][i] == d['filename'][inds[0]]){
            same_file.push(i)
        }
    }
    
    for (var i = 0; i < same_file.length; i++){
        d['color'][same_file[i]] = "firebrick"
    }

    source.selected.indices = same_file
    source.change.emit();
""")
source.selected.js_on_change('indices', cb_click)

## Tool
plot.add_tools(BoxZoomTool())
plot.add_tools(hover)
plot.add_tools(PanTool())
plot.add_tools(ResetTool())
plot.add_tools(SaveTool())
plot.add_tools(TapTool(callback=cb_click))
plot.add_tools(WheelZoomTool())

curdoc().add_root(plot)

show(plot)

In [ ]:
from random import random
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
# output_notebook()


x = [random() for x in range(500)]
y = [random() for y in range(500)]
color = ["navy"] * len(x)

s = ColumnDataSource(data=dict(x=x, y=y, color=color))
p = figure(plot_width=400, plot_height=400, tools="tap", title="Select Here")
p.circle('x', 'y', color='color', size=8, source=s, alpha=0.4)

s2 = ColumnDataSource(data=dict(x=[0, 1], ym=[0.5, 0.5]))
p.line(x='x', y='ym', color="orange", line_width=5, alpha=0.6, source=s2)

# Tap Tool selector
s.selected.js_on_change('indices', CustomJS(args=dict(s=s), code="""
    const inds = s.selected.indices;
    const d = s.data;
    var ym = 0
    
    console.log(s)
    
    for (var i = 0; i < d['color'].length; i++){
        d['color'][i] = "navy"
    }
    
    if (inds.length == 0)
        return;
        
    nearby_pts = []
    for (var i = 0; i < d['color'].length; i++){
        if (d['y'][i] - d['y'][inds[0]] < 0.05 && d['y'][inds[0]] - d['y'][i] < 0.05){
            nearby_pts.push(i)
        }
    }
    
    for (var i = 0; i < nearby_pts.length; i++){
        d['color'][nearby_pts[i]] = "firebrick"
    }

    s.selected.indices = nearby_pts
    s.change.emit();
"""))

show(p)